# 2-clean&filter

In [10]:
import pandas as pd

df = pd.read_csv(
    "../data/interim/extract_16.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(337030, 32)

In [11]:
# Ne garder que les prises de parole et les interruptions
df = df[
    (df["Code_grammaire"] == "PAROLE_GENERIQUE")
    | (df["Code_grammaire"] == "INTERRUPTION_1_10")
]  # ou : df["Code_grammaire"].str.strip().isin(["PAROLE_GENERIQUE", "INTERRUPTION_1_10"])

# Exclure les prises de parole de "Mme la présidente" et "M. le président"
# Role_debat n'est pas bien identifié, utiliser Nom_orateur
df = df[~df["Nom_orateur"].str.strip().isin(["M. le président", "Mme la présidente"])]

# Pour le sous cas de la 16 législature : nettoyer le fichier qui n'est pas au bon endroit
# = date de 2021
df = df[df["UID"] != "CRSANR5L16S2021O1N144"]

# Garder une trace de la longueur des interventions brutes
df["len_dirtytext"] = df["Texte"].str.len()

# Stabiliser le ID_orateur pour etre au format AN (pour matcher données)
df["ID_orateur"] = "PA" + df["ID_orateur"]

# Changer les missing values pour non_précisé (majoritaire) dans Code_parole
df["Code_parole"] = df["Code_parole"].fillna("non_précisé")

df.shape

(215127, 33)

In [12]:
# aperçu des répartitions
df.groupby("Code_parole", dropna=False)["len_dirtytext"].describe()

,count,mean,std,min,25%,50%,75%,max
Code_parole,,,,,,,,
AVIS_COM_1_10,1.0,1278.000000,NaN,1278.0,1278.0,1278.0,1278.00,1278.0
AVIS_COM_1_20,10951.0,389.245457,416.340444,4.0,96.0,273.0,538.00,5395.0
AVIS_GVT_1_20,9954.0,418.768033,581.874887,4.0,20.0,212.0,581.75,6481.0
PAROLE_1_1,1.0,32.000000,NaN,32.0,32.0,32.0,32.00,32.0
PAROLE_1_2,70542.0,818.220961,1050.016891,3.0,221.0,463.0,999.00,21406.0
non_précisé,123678.0,257.965313,570.433599,3.0,18.0,38.0,195.00,16991.0


## Match députés

In [13]:
df_deputes = pd.read_csv("../data/raw/id-dep/deputes-historique(datan-datagouv).csv")

In [14]:
# shape avant fusion
df.shape

(215127, 33)

In [15]:
# Join des données sur la base de l'id des intervenants
df = df.join(df_deputes.set_index("id"), on="ID_orateur", how="left", rsuffix="_dep")

In [16]:
# shape après fusion
df.shape

(215127, 59)

## Export

In [17]:
# certaines col du join introduisent une erreur à l'import/export
# forcer le QUOTE_ALL permet de résoudre

import csv  # pour résoudre le soucis d'écart. Checker

df.to_csv(
    "../data/interim/data_cleaning.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [18]:
# verif excriture/lecture ok
df_test = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df_test.shape

(215127, 59)